<a href="https://colab.research.google.com/github/fatema373/NLP/blob/main/Text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
nltk.download("stopwords")
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install --upgrade gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install country_list

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
from os import listdir
from os.path import isfile, join
import pandas as pd
import string
import nltk
import re
import urllib,tarfile
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer,WordNetLemmatizer
from country_list import countries_for_language
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from gensim.models.word2vec import Word2Vec
import gensim.downloader as api
import gensim
import numpy
import sys
from keras.models import *
from keras.layers import *
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from keras import *
import keras

In [ ]:
!wget http://wstyler.ucsd.edu/files/enronsentv1.tar.gz

--2023-05-21 19:58:43--  http://wstyler.ucsd.edu/files/enronsentv1.tar.gz
Resolving wstyler.ucsd.edu (wstyler.ucsd.edu)... 132.239.147.75
Connecting to wstyler.ucsd.edu (wstyler.ucsd.edu)|132.239.147.75|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://wstyler.ucsd.edu/files/enronsentv1.tar.gz [following]
--2023-05-21 19:58:43--  https://wstyler.ucsd.edu/files/enronsentv1.tar.gz
Connecting to wstyler.ucsd.edu (wstyler.ucsd.edu)|132.239.147.75|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26406491 (25M) [application/x-gzip]
Saving to: ‘enronsentv1.tar.gz.1’

enronsentv1.tar.gz. 100%[===================>]  25.18M  2.50MB/s    in 10s     

2023-05-21 19:58:54 (2.49 MB/s) - ‘enronsentv1.tar.gz.1’ saved [26406491/26406491]



In [ ]:
with tarfile.open("/content/enronsentv1.tar.gz", 'r:gz') as tar:
    tar.extractall()

In [ ]:
# path = '/content/enronsent'
# paths = list(set(os.listdir(path)))
# df = pd.DataFrame()
# df['docs']=[read_file(os.path.join(path, x)) for x in os.listdir(path) if 'enronsent' in x and '._enronsent' not in x]
# df['file_name']= [int(x[-6:-4]) for x in os.listdir(path) if 'enronsent' in x and '._enronsent' not in x]

In [ ]:
class DataLoader:
    def __init__(self,originpath):
        self.originpath=originpath
        self.df=pd.DataFrame()
        # save the docs and their number
        # the files is named as following enronsentXX, the XX is representing the no of doc
        #  so we extract these two indecies and add them to be the df indexing in the future
        self.df['origin_doc']=self.read_docs(self.originpath)
        self.preprocessing()
        self.df=self.padding(self.df,self.tokenizer)
    def padding(df,tokenizer):
        max_length = 15
        def sequences(p):
            return tokenizer.texts_to_sequences(p)
        df['padding'] = df.clean_text.apply(lambda p: pad_sequences(sequences(p), maxlen=max_length, padding='post'))
        return df
    def read_docs(self,path):
        docs=[]
        for doc in os.listdir(path):
            doc_path=os.path.join(path,doc)
            with open(doc_path,encoding='utf-8',errors='ignore') as file:
                docs.append(file.read().replace('\n', ''))
        return docs

    def change_file_extension(path):
        for file in os.listdir(path):
            # print (file)
            head, tail = os.path.splitext(file)
            if not tail:
                # print('fdf')
                dst = os.path.join(path, head + '.txt')
                src = os.path.join(path, head)
                new_name = head + '.txt'
                new_filepath = os.path.join(path, new_name)
                os.rename(src, new_filepath)
    def preprocessing(self):
        # sentence_end_regex = r"[.!?]" #convert docs to sentences
        # self.df['sentences']=self.df['origin_doc'].apply(lambda x : list(filter(None,list(re.split(sentence_end_regex, x)))))

        pretext=[]
        for doc in self.df['origin_doc']:# remove chars and puncs
            punctuations = string.punctuation
            pretext.append(doc.translate(str.maketrans('','',punctuations)))

        self.df['pre_text']= pretext
        punctuations = string.punctuation

        stop_words = set(stopwords.words('english')) # remove stop words
        pretext=[]
        for doc in self.df.pre_text:
            pretext.append(' '.join(word for word in doc.split() if word not in stop_words))
        self.df['pre_text']= pretext
        pretext=[]
        for doc in  self.df['pre_text']:
            doc = re.sub('[^a-zA-Z]',' ',doc)
            doc = re.sub('\s+',' ',doc)
            pretext.append(doc)
        self.df['pre_text']= pretext

        pretext=[]
        for doc in  self.df['pre_text']:
            countries = dict(countries_for_language('en'))
            countries_Vals = [x.lower() for x in list(countries.values())]
            countries_Keys = [x.lower() for x in list(countries.keys())]
            countries_to_be_removed=countries_Vals+countries_Keys
            pretext.append(' '.join(word for word in doc.split() if word not in (countries_to_be_removed)))
        self.df['pre_text']= pretext

        months=['january','february','march','april', 'may','june','july','august','september','october','november','december']
        pretext=[]
        for doc in self.df['pre_text']:
            pretext.append(' '.join(word for word in doc.split() if word not in (months)))
        self.df['pre_text']= pretext
        self.df.pre_text = df.pre_text.apply(lambda x : list(filter(None,x))) #remove nanas

        X=[]
        for text in df.pre_text:
            X.extend(text)
            tokenizer = Tokenizer()
            tokenizer.fit_on_texts(X)
        self.tokenizer=tokenizer
    def get_data(self):
        return self.df

In [ ]:
path = '/content/enronsent'
dataLoader=DataLoader(path)

In [ ]:
df=dataLoader.df

In [ ]:
model = gensim.models.KeyedVectors.load_word2vec_format('/content/drive/MyDrive/GoogleNews-vectors-negative300.bin.gz', binary=True)

In [ ]:
model.vector_size

300

In [ ]:
tokenizer=dataLoader.tokenizer

In [ ]:
num_words = len(tokenizer.word_index) + 1
embedding_dim = 300
embedding_matrix = np.zeros((num_words, embedding_dim))
for word, i in tokenizer.word_index.items():
    if word in model:
        embedding_matrix[i] = model[word]

In [ ]:
train =np.array(df.padding.iloc[:40])
validation = np.array(df.padding.iloc[40:])

In [ ]:
import numpy as np

In [ ]:
train =np.array(df.padding.iloc[:11])
validation = np.array(df.padding.iloc[11:])

In [ ]:
def create_batch(X_, n_steps, tokenizer):
  X, y = [],[]
  for i in range(len(X_)-n_steps-1):
    words = tokenizer.sequences_to_texts([[i] for i in X_[i:i+n_steps]])
    words = [word for word in words if len(word)>0]
    if len(words)==0:
      return [],[]
    X.append(model[words])
    y.append(model[tokenizer.sequences_to_texts([[X_[i+n_steps]]])])
  X=np.array(X)
  return X, np.array(y)

In [ ]:
def timeStep_prep(data,timestep=1):
    X = list()
    y = list()
    for i in range(timestep, data.shape[0]):
        step = np.zeros((timestep, data.shape[0]-1))
        for j in reversed(range(1,timestep+1)):
            step[timestep-j] = data[i-j]
        y.append(data[i])
        X.append(step)
    return np.array([np.array(X), np.array(y)])

In [ ]:
# # num_words = len(tokenizer.word_index) + 1
# # embedding_dim = 300
# # max_length=15
from tensorflow.keras.callbacks import EarlyStopping
early_stop_LSTM = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min', restore_best_weights=True)

# embedding_layer = Embedding(num_words, embedding_dim, weights=[embedding_matrix], input_length=max_length, trainable=False)
LSTM_model = Sequential()
LSTM_model.add(Input((4,300)))
# LSTM_model.add(embedding_layer)
LSTM_model.add(LSTM(4,activation="relu",dropout=0.3))
LSTM_model.add(BatchNormalization())
LSTM_model.add(Dense(512,activation='relu'))
LSTM_model.add(Dense(512,activation='relu'))
LSTM_model.add(Dense(256,activation='relu'))
LSTM_model.add(Dense(len(b),activation='softmax'))
optimizer = keras.optimizers.Adam(lr=0.3)

LSTM_model.compile(loss='categorical_crossentropy', optimizer=optimizer,metrics=['categorical_crossentropy'])

LSTM_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 4)                 4880      
                                                                 
 batch_normalization (BatchN  (None, 4)                16        
 ormalization)                                                   
                                                                 
 dense (Dense)               (None, 512)               2560      
                                                                 
 dense_1 (Dense)             (None, 512)               262656    
                                                                 
 dense_2 (Dense)             (None, 256)               131328    
                                                                 
 dense_3 (Dense)             (None, 297569)            76475233  
                                                        

/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [ ]:
data=[create_batch(sc,4) for doc in train for sc in doc if len(create_batch(sc,4))==2 ]

In [ ]:
from keras.preprocessing.sequence import TimeseriesGenerator
generator = TimeseriesGenerator(train, train, length=15, batch_size=4)

In [ ]:
generator[1]